## Leer data de una Base de Datos

### Cargo Paquetes Iniciales

In [3]:
from functions.create_db import load_data, dict_Empresas, write_sql_main
import pandas as pd
import datetime as dt
import sqlite3

### Cargo Datos (Clase Anterior)

In [4]:
df = load_data(dict_Empresas, start=dt.date(2024,1,1))

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Archivo Apple cargado exitosamente.


[*********************100%***********************]  1 of 1 completed


Archivo Microsoft cargado exitosamente.


[*********************100%***********************]  1 of 1 completed


Archivo Meta cargado exitosamente.


[*********************100%***********************]  1 of 1 completed

Archivo Tesla cargado exitosamente.


In [5]:
df

,Date,Close,High,Low,Open,Volume,Empresa,TKT
0,2024-01-02,184.290405,187.070052,182.553128,185.789422,82488700,Apple,AAPL
1,2024-01-03,182.910538,184.528693,182.096492,182.880757,58414500,Apple,AAPL
2,2024-01-04,180.587540,181.758954,179.565029,180.825785,71983600,Apple,AAPL
3,2024-01-05,179.862823,181.431339,178.860172,180.666948,62303300,Apple,AAPL
4,2024-01-08,184.210983,184.250701,180.180502,180.766209,59144500,Apple,AAPL
...,...,...,...,...,...,...,...,...
349,2025-05-23,339.339996,343.179993,333.209991,337.920013,84654800,Tesla,TSLA
350,2025-05-27,362.890015,363.790009,347.320007,347.350006,120146400,Tesla,TSLA
351,2025-05-28,356.899994,365.000000,355.910004,364.839996,91404300,Tesla,TSLA
352,2025-05-29,358.429993,367.709991,356.000000,365.290009,88545700,Tesla,TSLA


In [7]:
write_sql_main(DB='../local_2/database/stock_db.db',
               df = df,
               table="main")

Connected to SQLite
The SQLite connection is closed


'ok'

### Leer Info de una Base de Datos

In [8]:
#Me conecto a la Base de Datos
conn = sqlite3.connect('../local/database/stock_db.db')

In [9]:
#¿Como ver las tablas de una BD?
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", conn)
tables.head()

,name
0,main


In [10]:
#¿Como ver las tabla de compra_ventas?
main_table = pd.read_sql_query("SELECT * FROM main", conn)
main_table.head()

,Date,Close,High,Low,Open,Volume,Empresa,TKT
0,2024-01-02 00:00:00,184.290405,187.070052,182.553128,185.789422,82488700,Apple,AAPL
1,2024-01-03 00:00:00,182.910538,184.528693,182.096492,182.880757,58414500,Apple,AAPL
2,2024-01-04 00:00:00,180.587540,181.758954,179.565029,180.825785,71983600,Apple,AAPL
3,2024-01-05 00:00:00,179.862823,181.431339,178.860172,180.666948,62303300,Apple,AAPL
4,2024-01-08 00:00:00,184.210983,184.250701,180.180502,180.766209,59144500,Apple,AAPL


In [11]:
conn.close()

## Export to csv

In [12]:
main_table.to_csv('../local/csvs/stock_db.csv')

## Conexión con BigQuery

In [13]:
# Generamos conexion a BQ
from google.oauth2 import service_account # para generar conexion con BigQuery
#bq_cred = service_account.Credentials.from_service_account_file('../local/tokens/gcp_token2.json')
bq_cred = service_account.Credentials.from_service_account_file('../local/tokens/gcp_token_clase.json')

bq_cred

In [14]:
# Vemos que hay en "main_table"
main_table = pd.read_csv('../local/csvs/stock_db.csv')
main_table = main_table[['Date', 'Close', 'High', 'Low', 'Open', 'Volume', 'Empresa', 'TKT']]
main_table.head()

,Date,Close,High,Low,Open,Volume,Empresa,TKT
0,2024-01-02 00:00:00,184.290405,187.070052,182.553128,185.789422,82488700,Apple,AAPL
1,2024-01-03 00:00:00,182.910538,184.528693,182.096492,182.880757,58414500,Apple,AAPL
2,2024-01-04 00:00:00,180.587540,181.758954,179.565029,180.825785,71983600,Apple,AAPL
3,2024-01-05 00:00:00,179.862823,181.431339,178.860172,180.666948,62303300,Apple,AAPL
4,2024-01-08 00:00:00,184.210983,184.250701,180.180502,180.766209,59144500,Apple,AAPL


### Cargar Datos en BQ

In [15]:
# Enviamos el df a BQ - Podemos crear o reemplazar una tabla
import pandas_gbq # para interactuar con BQ

pandas_gbq.to_gbq(main_table, 'ucema_2025.main_from_vs_2', project_id= 'ucema-461612',
                   if_exists= 'replace', credentials = bq_cred)

In [8]:
# Si queremos anexar datos de una tabla en lugar de reemplazar
pandas_gbq.to_gbq(main_table, 'ucema_2025.main_from_vs', project_id= 'ucema-461612',
                   if_exists= 'append', credentials = bq_cred)

In [7]:
# Renombramos la columna "Adj Close" a "Adj_Close"
main_table = main_table.rename(columns={'Adj Close': 'Adj_Close'})

pandas_gbq.to_gbq(main_table, 'ucema_2025.main_from_vs', project_id= 'ucema-461612',
                   if_exists= 'replace', credentials = bq_cred)

### Leer Datos de BQ

In [12]:
# Si queremos leer una tabla
sql = """SELECT * FROM `ucema-461612.ucema_2025.main_from_vs`"""
df_bq = pd.read_gbq(sql, project_id='ucema-461612', credentials = bq_cred, dialect='standard') 

df_bq.head()

/var/folders/7_/kmrlbc594d903fc6dhmzxdrm0000gn/T/ipykernel_34170/3397499726.py:3: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df_bq = pd.read_gbq(sql, project_id='ucema-461612', credentials = bq_cred, dialect='standard')


,Date,Close,High,Low,Open,Volume,Empresa,TKT
0,2023-01-03 00:00:00,123.470619,129.226060,122.582127,128.613993,112117500,Apple,AAPL
1,2023-01-04 00:00:00,124.744125,127.014716,123.480495,125.267347,89113600,Apple,AAPL
2,2023-01-05 00:00:00,123.421249,126.136083,123.164580,125.504267,80962700,Apple,AAPL
3,2023-01-06 00:00:00,127.962433,128.623863,123.292924,124.398604,87754700,Apple,AAPL
4,2023-01-09 00:00:00,128.485687,131.704009,128.229017,128.801602,70790800,Apple,AAPL


In [13]:
df_bq.shape

(4760, 8)